# Lab 11 - Module 3: Revise & Compare (Stage 3)

**Time:** ~20-25 minutes

## Stage 3: Revision and Improvement

In Module 2, you identified the 2 weakest criteria in the AI's output. Now you'll test whether AI can **improve when given explicit feedback**.

### Learning Objectives

- Use rubric feedback to guide AI revision
- Evaluate whether explicit criteria help AI improve
- Identify trade-offs in revision (gains vs. new problems)
- Compare original vs. revised quality systematically
- Understand AI as a revision tool (not an authority)

### What You'll Do

1. Review the 2 weakest criteria from Module 2
2. Create a targeted revision prompt
3. Get the AI to revise its work
4. Score the revised version using the same rubric
5. Compare: Did it improve? Where? Did new problems appear?
6. Make an overall judgment: original or revised?

## Setup: Load Your Data

In [ ]:
import numpy as np
import pandas as pd
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown

print("✓ Libraries loaded!")

In [ ]:
# Load all previous data
group_code = int(input("Enter your group code: "))

try:
    with open(f'lab11_group_{group_code}_scenario.json', 'r') as f:
        scenario = json.load(f)
    
    with open(f'lab11_group_{group_code}_module1.json', 'r') as f:
        module1_data = json.load(f)
    
    with open(f'lab11_group_{group_code}_module2.json', 'r') as f:
        module2_data = json.load(f)
    
    print(f"✓ Loaded all data for Group {group_code}")
    print(f"\nPrompt Family: {scenario['family']}")
    print(f"Rubric Used: {module2_data['selected_rubric']}")
    print(f"\nWeakest Criteria Identified:")
    print(f"  1. {module2_data['weakest_criteria'][0]}")
    print(f"  2. {module2_data['weakest_criteria'][1]}")
    
except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find required files for group {group_code}.")
    print("Please run Modules 0-2 first.")

## Review: Original Output and Scores

In [ ]:
print("="*70)
print("ORIGINAL AI OUTPUT")
print("="*70)
print()
print(module1_data['ai_output'])
print()
print("="*70)
print("\nYOUR ORIGINAL SCORES:")
print("="*70)
for criterion, score in module2_data['human_scores'].items():
    marker = "⚠️ WEAK" if criterion in module2_data['weakest_criteria'] else ""
    print(f"  {criterion}: {score}/5 {marker}")
print("="*70)

## Step 1: Create a Revision Prompt

You'll ask the AI to revise its work, focusing specifically on the 2 weakest criteria.

### Effective Revision Prompts:

**Good approach:**
- Be specific about what to improve
- Reference the rubric criteria by name
- Give concrete examples of what's missing

**Less effective:**
- "Make it better"
- "Fix the problems"
- Vague, non-actionable feedback

In [ ]:
# Auto-generate suggested revision prompt
def generate_revision_prompt_template():
    weak_1 = module2_data['weakest_criteria'][0]
    weak_2 = module2_data['weakest_criteria'][1]
    
    template = f"""Please revise your previous response to improve these two specific areas:

1. **{weak_1}**: [Explain specifically what needs improvement based on the rubric]

2. **{weak_2}**: [Explain specifically what needs improvement based on the rubric]

Keep the same length (~same word count) and maintain the strengths of your original response, but focus on addressing these two weaknesses.

Provide the complete revised version."""
    
    return template

# Display template
template_button = widgets.Button(
    description='📝 Show Template',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

template_output = widgets.Output()

def on_template_clicked(b):
    with template_output:
        clear_output()
        print("SUGGESTED REVISION PROMPT TEMPLATE")
        print("="*70)
        print("(Fill in the [brackets] with specific feedback)")
        print()
        print(generate_revision_prompt_template())
        print()
        print("="*70)

template_button.on_click(on_template_clicked)

display(template_button)
display(template_output)

In [ ]:
# Widget to write custom revision prompt
revision_prompt = widgets.Textarea(
    value='',
    placeholder='Write your revision prompt here. Be specific about what needs improvement.',
    description='Revision Prompt:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': '120px'}
)

print("\nWRITE YOUR REVISION PROMPT")
print("="*70)
print("This is what you'll give to the AI to request a revision.")
print()
display(revision_prompt)

## Step 2: Get the Revised Version from AI

### Instructions:

1. Go back to your AI conversation
2. Copy your revision prompt from above
3. Paste it into the AI
4. Wait for the complete revised response
5. Copy the ENTIRE revised output
6. Paste it below

In [ ]:
# Widget to record revised output
revised_output = widgets.Textarea(
    value='',
    placeholder='Paste the complete REVISED output from the AI here...',
    description='Revised Output:',
    layout=widgets.Layout(width='100%', height='300px'),
    style={'description_width': '120px'}
)

ai_change_summary = widgets.Textarea(
    value='',
    placeholder='Did the AI explain what it changed? Record that here.',
    description='AI Changes:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '120px'}
)

print("RECORD REVISED OUTPUT")
print("="*70)
display(revised_output)
display(ai_change_summary)

## Step 3: Score the Revised Version

Now evaluate the **revised** output using the **same rubric** you used in Module 2.

In [ ]:
# Load rubric definition
RUBRICS = {
    'General Communication': {
        'criteria': [
            {'name': 'Clarity'},
            {'name': 'Audience Fit'},
            {'name': 'Structure'},
            {'name': 'Specificity'},
            {'name': 'Accuracy'}
        ]
    },
    'Persuasion/Campaign': {
        'criteria': [
            {'name': 'Message Focus'},
            {'name': 'Evidence/Support'},
            {'name': 'Call to Action'},
            {'name': 'Tone Fit'},
            {'name': 'Ethics'}
        ]
    },
    'Creative/Narrative': {
        'criteria': [
            {'name': 'Engagement'},
            {'name': 'Coherence'},
            {'name': 'Creativity'},
            {'name': 'Constraint Adherence'}
        ]
    }
}

# Create scoring widgets for revised version
selected_rubric = RUBRICS[module2_data['selected_rubric']]
revised_scores = {}
revised_justifications = {}

print("SCORE THE REVISED VERSION")
print("="*70)
print(f"Using the same rubric: {module2_data['selected_rubric']}")
print()

for criterion in selected_rubric['criteria']:
    name = criterion['name']
    original_score = module2_data['human_scores'][name]
    
    score_widget = widgets.Dropdown(
        options=[1, 2, 3, 4, 5],
        description=f"{name}:",
        style={'description_width': '150px'},
        layout=widgets.Layout(width='300px')
    )
    
    justification_widget = widgets.Textarea(
        value='',
        placeholder=f'Why this score? Did it improve from {original_score}?',
        layout=widgets.Layout(width='100%', height='60px')
    )
    
    revised_scores[name] = score_widget
    revised_justifications[name] = justification_widget
    
    marker = "⚠️ TARGET" if name in module2_data['weakest_criteria'] else ""
    print(f"Original score: {original_score}/5 {marker}")
    display(score_widget)
    display(justification_widget)
    print()

## Step 4: Compare Original vs. Revised

In [ ]:
# Generate comparison
compare_button = widgets.Button(
    description='📊 Compare Versions',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

compare_output = widgets.Output()

def on_compare_clicked(b):
    with compare_output:
        clear_output()
        
        # Build comparison table
        comparison_data = []
        total_original = 0
        total_revised = 0
        improvements = []
        regressions = []
        
        for criterion in selected_rubric['criteria']:
            name = criterion['name']
            original_score = module2_data['human_scores'][name]
            revised_score = revised_scores[name].value
            change = revised_score - original_score
            
            total_original += original_score
            total_revised += revised_score
            
            if change > 0:
                improvements.append((name, original_score, revised_score, change))
                change_str = f"✓ +{change}"
            elif change < 0:
                regressions.append((name, original_score, revised_score, change))
                change_str = f"✗ {change}"
            else:
                change_str = "→ Same"
            
            target = "🎯" if name in module2_data['weakest_criteria'] else ""
            
            comparison_data.append({
                'Criterion': name,
                'Original': original_score,
                'Revised': revised_score,
                'Change': change_str,
                'Target': target
            })
        
        df = pd.DataFrame(comparison_data)
        
        print("="*70)
        print("ORIGINAL VS. REVISED COMPARISON")
        print("="*70)
        print()
        print(df.to_string(index=False))
        print()
        print("="*70)
        print(f"Total Original Score: {total_original}/{len(selected_rubric['criteria']) * 5}")
        print(f"Total Revised Score:  {total_revised}/{len(selected_rubric['criteria']) * 5}")
        print(f"Overall Change:       {total_revised - total_original:+d}")
        print()
        
        if improvements:
            print("IMPROVEMENTS:")
            for name, orig, rev, change in improvements:
                target_marker = " (TARGET)" if name in module2_data['weakest_criteria'] else ""
                print(f"  ✓ {name}: {orig} → {rev} (+{change}){target_marker}")
            print()
        
        if regressions:
            print("REGRESSIONS (Got worse):")
            for name, orig, rev, change in regressions:
                print(f"  ✗ {name}: {orig} → {rev} ({change})")
            print()
        
        if not improvements and not regressions:
            print("No changes in any criterion.")
            print()
        
        # Check if targeted weaknesses improved
        weak_1 = module2_data['weakest_criteria'][0]
        weak_2 = module2_data['weakest_criteria'][1]
        weak_1_change = revised_scores[weak_1].value - module2_data['human_scores'][weak_1]
        weak_2_change = revised_scores[weak_2].value - module2_data['human_scores'][weak_2]
        
        print("TARGETED IMPROVEMENTS:")
        print(f"  {weak_1}: {weak_1_change:+d} {'✓' if weak_1_change > 0 else '✗' if weak_1_change < 0 else '→'}")
        print(f"  {weak_2}: {weak_2_change:+d} {'✓' if weak_2_change > 0 else '✗' if weak_2_change < 0 else '→'}")
        print()
        print("="*70)

compare_button.on_click(on_compare_clicked)

display(compare_button)
display(compare_output)

## Step 5: Analysis Questions

In [ ]:
# Analysis widgets
new_problems = widgets.Textarea(
    value='',
    placeholder='Did revision introduce any NEW problems? Be specific.',
    description='New Problems:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '120px'}
)

overall_judgment = widgets.RadioButtons(
    options=['Original is better', 'Revised is better', 'About the same', 'Mixed (some better, some worse)'],
    description='Overall:',
    style={'description_width': '120px'}
)

judgment_reasoning = widgets.Textarea(
    value='',
    placeholder='Why did you choose this judgment? Consider all criteria, not just the targeted ones.',
    description='Reasoning:',
    layout=widgets.Layout(width='100%', height='120px'),
    style={'description_width': '120px'}
)

further_revision = widgets.Textarea(
    value='',
    placeholder='If you could revise again, what would you focus on?',
    description='Next Revision:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '120px'}
)

print("\nANALYSIS")
print("="*70)
display(new_problems)
display(overall_judgment)
display(judgment_reasoning)
display(further_revision)

## Step 6 (Optional): Ask AI to Score the Revision

If time permits, you can ask the AI to score its **revised** version and see if its self-evaluation improved.

In [ ]:
# Optional: AI self-eval of revision
ai_revised_self_eval = widgets.Textarea(
    value='',
    placeholder='(Optional) Paste AI\'s self-evaluation of the REVISED version here',
    description='AI Rev. Eval:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': '120px'}
)

ai_eval_comparison = widgets.Textarea(
    value='',
    placeholder='(Optional) Was AI more/less accurate in evaluating the revision?',
    description='Comparison:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '120px'}
)

print("\nOPTIONAL: AI SELF-EVALUATION OF REVISION")
print("="*70)
display(ai_revised_self_eval)
display(ai_eval_comparison)

## Step 7: Save All Data

In [ ]:
# Save button
save_module3_button = widgets.Button(
    description='💾 Save Module 3',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

save_output = widgets.Output()

def on_save_module3_clicked(b):
    with save_output:
        clear_output()
        
        # Validate
        if not revision_prompt.value:
            print("❌ Please write a revision prompt")
            return
        if not revised_output.value or len(revised_output.value.strip()) < 50:
            print("❌ Please paste the revised output")
            return
        if not overall_judgment.value:
            print("❌ Please make an overall judgment")
            return
        
        # Compile data
        revised_scores_data = {name: widget.value for name, widget in revised_scores.items()}
        revised_justifications_data = {name: widget.value for name, widget in revised_justifications.items()}
        
        module3_data = {
            'group_code': group_code,
            'revision_prompt': revision_prompt.value,
            'revised_output': revised_output.value,
            'ai_change_summary': ai_change_summary.value,
            'revised_scores': revised_scores_data,
            'revised_justifications': revised_justifications_data,
            'new_problems': new_problems.value,
            'overall_judgment': overall_judgment.value,
            'judgment_reasoning': judgment_reasoning.value,
            'further_revision_ideas': further_revision.value,
            'ai_revised_self_eval': ai_revised_self_eval.value,
            'ai_eval_comparison': ai_eval_comparison.value
        }
        
        # Save
        with open(f'lab11_group_{group_code}_module3.json', 'w') as f:
            json.dump(module3_data, f, indent=2)
        
        print("✓ Module 3 data saved successfully!")
        print(f"\nOverall Judgment: {overall_judgment.value}")
        print(f"\nYou're ready for Module 4 (Reflection & Synthesis)!")

save_module3_button.on_click(on_save_module3_clicked)

print("\n" + "="*70)
print("SAVE YOUR WORK")
print("="*70)
display(save_module3_button)
display(save_output)

## Module 3 Questions

Answer these on your Lab 11 Answer Sheet.

### Q12: Revision Prompt

What were the 2 weakest criteria? What revision prompt did you use? Was it specific and actionable?

*(Answer on your answer sheet)*

### Q13: Improvement Analysis

Did the revised version improve on the targeted criteria? Show the comparison (original score → revised score for each targeted criterion).

*(Answer on your answer sheet with comparison table)*

### Q14: New Problems

Did the revision introduce any NEW problems? Did improving one area make another area worse?

*(Answer on your answer sheet)*

### Q15: Overall Judgment

Overall, is the revised version better than the original? Why or why not? Consider ALL criteria, not just the targeted ones.

*(Answer on your answer sheet)*

### Q16 (Optional): AI Self-Evaluation Accuracy

If you asked AI to score its revision, was it more or less accurate than its original self-evaluation? Did it learn from the feedback?

*(Answer on your answer sheet)*

### Q17: Further Revision

If you had to revise AGAIN, what would you focus on? What's still weak?

*(Answer on your answer sheet)*

## Summary: What You've Accomplished

✓ Identified the 2 weakest criteria from Module 2

✓ Created a targeted revision prompt

✓ Got AI to revise its work

✓ Scored the revised version using the same rubric

✓ Compared original vs. revised systematically

✓ Identified improvements, regressions, and trade-offs

✓ Made an overall judgment about quality

### Key Insights

You've likely discovered:

1. **Explicit feedback helps**: When you give clear, rubric-based guidance, AI can often improve

2. **Trade-offs exist**: Improving one aspect sometimes makes another worse

3. **Iteration has limits**: AI revision is useful but not perfect

4. **Human judgment remains essential**: You, not the AI, must decide if the revision is actually better

### The Bigger Picture

AI is a **collaborator**, not an **authority**:
- ✓ Good at generating drafts
- ✓ Good at revising when given specific feedback
- ✗ Poor at evaluating its own quality
- ✗ Cannot reliably decide what's "good"

**Humans must provide the judgment, standards, and oversight.**

### What's Next?

In **Module 4 (Reflection)**, you will:
- Synthesize your findings from all 3 stages
- Read case studies about AI evaluation failures
- Articulate when AI is a good collaborator vs. when humans must lead
- Develop principles for responsible human-AI collaboration